In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from models.price_evaluator_decision_tree import (
    DecisionTreeHyperparams, 
    PriceClassifierBasicModel
)

from models.price_evaluator_xgboost_clasifier import (
    PriceClassifierXGBoostModelHyperparams,
    PriceClassifierXGBoostModel
)

from evaluation.evaluate_classification import evaluate_classification

from utils.load_dataset import load_dataset
from utils.load_hyperparameters import load_hyperparameters

In [ ]:
def plot_results(results):
    metrics = list(next(iter(results.values())).keys())
    model_names = list(results.keys())

    for metric in metrics:
        plt.figure()
        values = [results[model][metric] for model in model_names]
        plt.bar(model_names, values, color=['blue', 'orange'])
        plt.title(f"Comparison of {metric}")
        plt.ylabel(metric)
        plt.xlabel("Model")
        plt.show()

def main(hyperparameters_xgboost_path, hyperparameters_dt_path, dataset_path):
    X, y = load_dataset(dataset_path)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    xgboost_hyperparams_dict = load_hyperparameters(hyperparameters_xgboost_path)
    dt_hyperparams_dict = load_hyperparameters(hyperparameters_dt_path)

    xgboost_hyperparams = PriceClassifierXGBoostModelHyperparams(**xgboost_hyperparams_dict)
    dt_hyperparams = DecisionTreeHyperparams(**dt_hyperparams_dict)

    xgboost_model = PriceClassifierXGBoostModel(hyperparams=xgboost_hyperparams)
    dt_model = PriceClassifierBasicModel(hyperparams=dt_hyperparams)

    print("Training XGBoost model...")
    xgboost_model.fit(X_train, y_train)

    print("Training Decision Tree model...")
    dt_model.fit(X_train, y_train)

    print("Evaluating XGBoost model...")
    xgboost_predictions = xgboost_model.predict(X_test)
    xgboost_proba = xgboost_model.predict_proba(X_test)
    xgboost_results = evaluate_classification(xgboost_predictions, y_test, xgboost_proba)

    print("Evaluating Decision Tree model...")
    dt_predictions = dt_model.predict(X_test)
    dt_proba = dt_model.predict_proba(X_test)
    dt_results = evaluate_classification(dt_predictions, y_test, dt_proba)

    results = {
        "XGBoost": {
            "Accuracy": xgboost_results.accuracy,
            "Precision": xgboost_results.precision,
            "Recall": xgboost_results.recall,
            "F1": xgboost_results.f1,
            "ROC AUC": xgboost_results.roc_auc,
        },
        "Decision Tree": {
            "Accuracy": dt_results.accuracy,
            "Precision": dt_results.precision,
            "Recall": dt_results.recall,
            "F1": dt_results.f1,
            "ROC AUC": dt_results.roc_auc,
        },
    }

    for model, metrics in results.items():
        print(f"{model} Results:")
        for metric, value in metrics.items():
            print(f"  {metric}: {value:.4f}")
        print()

    plot_results(results)

In [ ]:
hyperparameters_xgboost_path = "path/to/xgboost_hyperparameters.json"
hyperparameters_dt_path = "path/to/dt_hyperparameters.json"
dataset_path = "data/Car_sale_ads.csv"

main(hyperparameters_xgboost_path, hyperparameters_dt_path, dataset_path)